In [ ]:
!pip install tensorflow

In [242]:
import pandas as pd
import numpy as np

import datetime 

import os
import tempfile
from scipy import stats
import tensorflow as tf

from google.cloud import storage

from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

from tensorflow.keras import Model, initializers, optimizers, regularizers
from tensorflow.keras.layers import Input, Dense, Conv1D, Dropout, LSTM, TimeDistributed, Activation,Softmax
from tensorflow.keras.callbacks import ModelCheckpoint

os.getcwd()





'/home/jupyter/physiolyx/src/old'

In [270]:
def sceneDict(table):
    '''add descr'''
    scene_dict = {0: 'Menu', 1: 'Goalkeeping', 2: 'Touch object', 3: 'Fruit picking', 4: 'Gym time trials',
                  5: 'Calibration', 6: 'Sorting'}

    for key in scene_dict.keys():
        table.loc[table.scene_index == key, 'scene_index'] = scene_dict[key]

    return table['scene_index']

def rotRescalerML(val, D=360):
    '''add descr'''
    return (np.where(val > D / 2, (val - D), val))


def valDivider(table, k=1000):
    '''add descr'''

    return table / k


def metscoreCalc(scene, table):
    '''add descr'''
    #0 - Menu
    #1 - Gym(goalkeeping)
    #2 - TouchObject
    #3 - Fruit picking
    #4 - Gym_TimeTrials(goalkeeping time trials)
    #5 - Calibration
    #6 - Sorting
    metscore_dict = {0: 1, 1: 2.5, 2: 2.5, 3: 3, 4: 2.5, 5: 1, 6: 2}

    metscore = []
    for row in range(len(table)):
        metscore.append(metscore_dict[table[scene][row]] / 4320) #60*72

    return metscore

def tableReader(file, cols):
    '''add descr'''
    if cols is None:
        cols = ['index', 'scene_index', 'time', 'ms_lastline', 'head_posx', "head_posy", "head_posz", "head_rotx",
            "head_roty", "head_rotz", "right_posx", "right_posy", "right_posz", "right_rotx", "right_roty",
            "right_rotz", "left_posx", "left_posy", "left_posz", "left_rotx", "left_roty", "left_rotz"]

    table = pd.read_table(file, sep=',', header=0, names=cols)

    return table


def tableProcessML(table, data):
    '''add descr'''
    '''data(dict): The Cloud Functions event payload.'''

    ### 2. divide /1000 ###
    colDivider = ['head_posx',"head_posy","head_posz","head_rotx","head_roty","head_rotz",
        "right_posx","right_posy","right_posz","right_rotx","right_roty","right_rotz",
        "left_posx","left_posy","left_posz","left_rotx","left_roty","left_rotz"]
    table.loc[:, colDivider] = valDivider(table.loc[:, colDivider])

    ### 3. format time ###
    table['time'] = pd.to_datetime(table['time'], format="%H:%M:%S").dt.time #do we really need this column?

    ### 4. create date ###
    table['date'] = pd.to_datetime(data['name'][12:22], format = "%d-%m-%Y")

    ### 5. create seconds ###
    #on hold for now
    #timedelta = pd.to_timedelta(table.time.astype(str))
    #diff = timedelta.diff().fillna(pd.Timedelta(seconds=0)) / 1e9
    #table['seconds'] = np.cumsum(diff).astype(int)

    ### 6. rescale rotations ###
    cols = ["head_rotx", "head_roty", "head_rotz", "right_rotx", "right_roty", "right_rotz",
            "left_rotx", "left_roty", "left_rotz"]

    for col in cols:
        table[col] = rotRescalerML(table[col])

    ### 7. compute MET score at frame level ###
    table['met_score'] = metscoreCalc('scene_index', table)

    ### 8. replace scene_index with string names ###
    table['scene_index'] = sceneDict(table)

    ### 9. create id column ###
    table['id'] = 1 #in production this should come from the quest

    ### 10. drop unused columns ###
    table = table.drop('index ms_lastline'.split(), axis=1)

    return table

def TrainPreprocess(df, cols, timeSteps=72, step=14, div=70, method='last'):
    '''preprocessing'''

    if cols is None:
        cols = ["head_rotx","head_roty","head_rotz","head_posx","head_posy","head_posz"]
        
    features= len(cols)
    segments = []
    labels = []

    print('Preprocessing started')
    
    df = df.fillna(0)
    
    #normalise data otherwise loss= nan
    for col in cols:
        df[col] = (df[col] - df[col].min())/(df[col].max()-df[col].min())

    #reshuffle?
    np.random.shuffle(df.values.reshape(-1,int(np.floor(df.shape[0]/div)),df.shape[1]))


    #we reshape into 3d arrays of length equal to timesteps. final df is= (N*timesteps*6)
    coldict = {}
    for i in range(0, len(df) - timeSteps, step):
        for col in cols:
            coldict[str(col[5:])] = table[col].values[i: i + timeSteps]
       
            segments.append(coldict[str(col[5:])])
        
        if method == 'max':
            label = stats.mode(df['action'][i:i + timeSteps])[0][0]
        else:
            label = df['action'].iloc[i + timeSteps]
        
        labels.append(label)
        
    reshaped_segments = np.asarray(segments, dtype= np.float32).reshape(-1, timeSteps, features)
    
    labels = pd.get_dummies(labels)
    truelabels= pd.get_dummies(labels).idxmax(1)
    labels = np.asarray(labels, dtype = np.float32)

    print('Preprocessing completed')

    return reshaped_segments, labels, truelabels


def download_blob(bucket_name, source_blob_name, destination_file_name):
    """Downloads a blob from the bucket."""
    client = storage.Client()
    bucket = client.get_bucket(bucket_name)
    blob = bucket.get_blob(source_blob_name)

    blob.download_to_filename(destination_file_name)

    print('Blob {} downloaded to {}.'.format(
        source_blob_name,
        destination_file_name))
    
    
def trainer(model, epochs=40, batch_size=12):
    '''removed .hdf5 from filename'''
    callbacks = ModelCheckpoint('model_ep{epoch:02d}_val{val_categorical_accuracy:.2f}', monitor='val_categorical_accuracy', verbose=0,
                                save_best_only=True, save_weights_only=True, mode='max')
    history = model.fit(X,y, validation_split = 0.2, batch_size = 12,
                epochs = epochs, callbacks = [callbacks])
    return history



def upload_blob(bucket, infile, outfile):
    '''add descr'''
    
    client = storage.Client()
    outbucket = client.get_bucket(bucket)
    outblob = outbucket.blob(outfile)
    
    with open(infile, "rb") as out:
        outblob.upload_from_file(out)
    
    print('File {} uploaded to {} as {}.'.format(
        infile, bucket, outfile))
    

    

In [244]:
colslab = ['index','scene_index','time','ms_lastline','head_posx',"head_posy","head_posz","head_rotx","head_roty","head_rotz",
        "right_posx","right_posy","right_posz","right_rotx","right_roty","right_rotz",
        "left_posx","left_posy","left_posz","left_rotx","left_roty","left_rotz",'timedel','action']

download_blob(data['bucket'],data['name'],'file.txt')
table = tableReader('file.txt',cols=colslab)


Blob moniLabData 20-03-2020.txt downloaded to file.txt.


In [245]:
table = tableProcessML(table,data)

In [246]:
table= table.loc[:,['action',"head_rotx","head_roty","head_rotz"]]

table = table[table.action != 'STILL']

table = table[0:11130]

In [247]:
table.groupby('action').size()

action
3DLEX     145
3DLFL     503
3DREX     361
3DRFL     359
EXTE     2230
FLEX     1799
LBEN     1010
LROT     1945
RBEN      864
RROT     1914
dtype: int64

## Control room

In [248]:
colspre = ["head_rotx","head_roty","head_rotz"]

timeSteps = 72
n_features = len(colspre)
n_classes = table.action.unique().size

method = 'last'

In [249]:
X,y, true = TrainPreprocess(table, cols=colspre, timeSteps=timeSteps, method = method)

Preprocessing started
Preprocessing completed


In [251]:
class DeepConvLSTM(Model):
  def __init__(self):
    super(DeepConvLSTM, self).__init__()
    self.c1 = Conv1D(8, 1,input_shape=(timeSteps, n_features), kernel_regularizer=regularizers.l2(0.02), activation='relu', kernel_initializer='orthogonal') #ordo filters=64, kernel_size = 5
    self.c2 = Conv1D(8, 3,kernel_regularizer=regularizers.l2(0.02), activation='relu', kernel_initializer='orthogonal')
    #self.c3 = Conv1D(8, 3,kernel_regularizer=regularizers.l2(0.02), activation='relu', kernel_initializer='orthogonal')
    #self.c4 = Conv1D(8, 3, activation='relu', kernel_initializer='orthogonal')
    self.do1 = Dropout(0.5)
    self.r1 = LSTM(16, activation='tanh', kernel_regularizer=regularizers.l2(0.02), return_sequences = True) #ordo cells=128
    self.do2 = Dropout(0.5)
    self.r2 = LSTM(16, activation='tanh', kernel_regularizer=regularizers.l2(0.02),  return_sequences = False)
    self.sm = Dense(n_classes, activation='softmax')
    
  def call(self, x):
    x = self.c1(x)
    x = self.c2(x)
    #x = self.c3(x)
    #x = self.c4(x)
    x = self.do1(x)
    x = self.r1(x)
    x = self.do2(x)
    x = self.r2(x)
    
    return self.sm(x)




In [253]:
class SimpleLSTM(Model):
  def __init__(self):
    super(SimpleLSTM, self).__init__()
    #self.c1 = Conv1D(8, 1, kernel_regularizer=regularizers.l2(0.02), activation='relu', kernel_initializer='orthogonal') #ordo filters=64, kernel_size = 5
    #self.c2 = Conv1D(8, 3,kernel_regularizer=regularizers.l2(0.02), activation='relu', kernel_initializer='orthogonal')
    #self.c3 = Conv1D(8, 3,kernel_regularizer=regularizers.l2(0.02), activation='relu', kernel_initializer='orthogonal')
    #self.c4 = Conv1D(8, 3, activation='relu', kernel_initializer='orthogonal')
    #self.do1 = Dropout(0.5)
    self.r1 = LSTM(32,input_shape=(timeSteps, n_features), activation='tanh', kernel_regularizer=regularizers.l2(0.02), return_sequences = True) #ordo cells=128
    #self.do2 = Dropout(0.5)
    self.r2 = LSTM(32, activation='tanh', kernel_regularizer=regularizers.l2(0.02),  return_sequences = False)
    self.sm = Dense(n_classes, activation='softmax')
    
  def call(self, x):
    #x = self.c1(x)
    #x = self.c2(x)
    #x = self.c3(x)
    #x = self.c4(x)
   # x = self.do1(x)
    x = self.r1(x)
    #x = self.do2(x)
    x = self.r2(x)
    
    return self.sm(x)





In [ ]:
class SimpleLSTM(Model):
  def __init__(self):
    super(SimpleLSTM, self).__init__()
    #self.c1 = Conv1D(8, 1, kernel_regularizer=regularizers.l2(0.02), activation='relu', kernel_initializer='orthogonal') #ordo filters=64, kernel_size = 5
    #self.c2 = Conv1D(8, 3,kernel_regularizer=regularizers.l2(0.02), activation='relu', kernel_initializer='orthogonal')
    #self.c3 = Conv1D(8, 3,kernel_regularizer=regularizers.l2(0.02), activation='relu', kernel_initializer='orthogonal')
    #self.c4 = Conv1D(8, 3, activation='relu', kernel_initializer='orthogonal')
    #self.do1 = Dropout(0.5)
    self.r1 = LSTM(32,input_shape=(timeSteps, n_features), activation='tanh', kernel_regularizer=regularizers.l2(0.02), return_sequences = True) #ordo cells=128
    #self.do2 = Dropout(0.5)
    self.r2 = LSTM(32, activation='tanh', kernel_regularizer=regularizers.l2(0.02),  return_sequences = False)
    self.sm = Dense(n_classes, activation='softmax')
    
  def call(self, x):
    #x = self.c1(x)
    #x = self.c2(x)
    #x = self.c3(x)
    #x = self.c4(x)
   # x = self.do1(x)
    x = self.r1(x)
    #x = self.do2(x)
    x = self.r2(x)
    
    return self.sm(x)



In [272]:
model = SimpleLSTM()
# define loss and optimizer
adam = optimizers.Adam(learning_rate=0.001) #(ordo learning_rate=0.01, decay=0.9)

model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['categorical_accuracy']) #or categorical_accuracy? #sparse categorical crossentropy if labels not one-hot encoded
# fit the model
#model.save_weights('model.h5')
model.save_weights('model.h5')

In [273]:
model.load_weights('model.h5')

trainer(model, epochs=20, batch_size=12)

Train on 632 samples, validate on 158 samples
Epoch 1/20
632/632 [==============================] - 7s 11ms/sample - loss: 3.0304 - categorical_accuracy: 0.2104 - val_loss: 2.7364 - val_categorical_accuracy: 0.2025
Epoch 2/20
632/632 [==============================] - 4s 6ms/sample - loss: 2.4680 - categorical_accuracy: 0.2484 - val_loss: 2.5318 - val_categorical_accuracy: 0.1962
Epoch 3/20
632/632 [==============================] - 4s 6ms/sample - loss: 2.2121 - categorical_accuracy: 0.2642 - val_loss: 2.4249 - val_categorical_accuracy: 0.2152
Epoch 4/20
632/632 [==============================] - 4s 6ms/sample - loss: 2.0928 - categorical_accuracy: 0.2737 - val_loss: 2.3883 - val_categorical_accuracy: 0.2468
Epoch 5/20
632/632 [==============================] - 4s 6ms/sample - loss: 1.9931 - categorical_accuracy: 0.3054 - val_loss: 2.3338 - val_categorical_accuracy: 0.2278
Epoch 6/20
632/632 [==============================] - 4s 6ms/sample - loss: 1.9535 - categorical_accuracy: 0.3038

In [274]:
weights=['model_ep04_val0.25.index','model_ep04_val0.25.data-00000-of-00001']

[upload_blob(data['bucket'],w,w) for w in weights]



File model_ep04_val0.25.index uploaded to physio-bucket as model_ep04_val0.25.index.
File model_ep04_val0.25.data-00000-of-00001 uploaded to physio-bucket as model_ep04_val0.25.data-00000-of-00001.


[None, None]

In [191]:
X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size= 0.3, random_state=1992)

(274, 200, 6)